In [196]:
from pyspark.sql import SparkSession
from pyspark.rdd import RDD
from pyspark.sql import Row, Window
from pyspark.sql import DataFrame
from pyspark.sql.functions import udf, lit
from pyspark.sql.functions import desc
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql.types import DecimalType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import when, col, lit
from pyspark.sql.functions import avg, stddev_pop

spark = SparkSession.builder.appName('ReadMariaDB') \
.config("spark.driver.memory", "32g") \
.config("spark.sql.pivotMaxValues", "1000000") \
.getOrCreate()

# sql = "select * from 01_sampled_games_2 WHERE playtime_forever IS NOT NULL AND playtime_forever > 0"
sql = "select * from 01_sampled_games_2 WHERE playtime_forever IS NOT NULL"
database = "steam"
user = "root"
password = "example"
server = "127.0.0.1"
port = 3306
jdbc_url = f"jdbc:mysql://{server}:{port}/{database}?permitMysqlScheme"
jdbc_driver = "org.mariadb.jdbc.Driver"

# Create a data frame by reading data from MariaDB via JDBC
df = spark.read.format("jdbc") \
    .option("url", jdbc_url) \
    .option("query", sql) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", jdbc_driver) \
    .load()

## Z-Score Normalization

In [197]:
from pyspark.sql.functions import mean, stddev_pop, col

# Calculate the per-game mean and standard deviation of the playtime column
game_stats = df.filter(col("playtime_forever") > 0).groupBy("appid").agg(
    mean("playtime_forever").alias("game_mean_playtime"),
    stddev_pop("playtime_forever").alias("game_stddev_playtime")
)

# Normalize the playtime column on a per-game basis using z-score normalization
df = df.join(game_stats, "appid")
df = df.withColumn("z_score", (col("playtime_forever") - col("game_mean_playtime")) / col("game_stddev_playtime"))
df = df.dropna(subset=["z_score"])

## Per-Game 1-5 Rank Normalization

First, we calculate the mean and standard deviation for the playtime for each user. Then, we define dynamically define the cut points for the ratings from 1 to 5 on a per-user basis.

In [198]:
df = df.withColumn("cut_point_1", when(col("game_mean_playtime") - (col("game_stddev_playtime") * 0.5) > 0, col("game_mean_playtime") - (col("game_stddev_playtime") * 0.5)).otherwise(0))
df = df.withColumn("cut_point_2", col("game_mean_playtime"))
df = df.withColumn("cut_point_3", col("game_mean_playtime") + (col("game_stddev_playtime") * 0.5))
df = df.withColumn("cut_point_4", col("game_mean_playtime") + col("game_stddev_playtime"))
df = df.withColumn("cut_point_5", col("game_mean_playtime") + (col("game_stddev_playtime") * 1.5))

Afterwards, we assign ratings for each playtime record based on the cut points.

In [199]:
df = df.withColumn(
    "ranks",
    when(col("playtime_forever") <= col("cut_point_1"), lit(1))
    .when((col("playtime_forever") > col("cut_point_1")) & (col("playtime_forever") <= col("cut_point_2")), lit(2))
    .when((col("playtime_forever") > col("cut_point_2")) & (col("playtime_forever") <= col("cut_point_3")), lit(3))
    .when((col("playtime_forever") > col("cut_point_3")) & (col("playtime_forever") <= col("cut_point_4")), lit(4))
    .otherwise(lit(5))
)

In [200]:
df.show()

+------+-----------------+---------------+----------------+-------------------+------------------+--------------------+--------------------+------------------+-----------+------------------+------------------+------------------+-----+
| appid|          steamid|playtime_2weeks|playtime_forever|      dateretrieved|game_mean_playtime|game_stddev_playtime|             z_score|       cut_point_1|cut_point_2|       cut_point_3|       cut_point_4|       cut_point_5|ranks|
+------+-----------------+---------------+----------------+-------------------+------------------+--------------------+--------------------+------------------+-----------+------------------+------------------+------------------+-----+
|221640|76561197960268000|           null|              27|2014-08-14 14:04:40|          186.0598|  361.95706223846577| -0.4394438362835636| 5.081268880767112|   186.0598| 367.0383311192329| 548.0168622384658| 728.9953933576986|    2|
|206480|76561197960268000|           null|             247|2

In [201]:
steamidIndexer = StringIndexer(inputCol="steamid", outputCol="steamidIndex")
model = steamidIndexer.fit(df)
df = model.transform(df)

appidIndexer = StringIndexer(inputCol="appid", outputCol="appidIndex")
model = appidIndexer.fit(df)
df = model.transform(df)

In [202]:
selected_rating_col = "ranks"
(training, test) = df.randomSplit([0.8, 0.2], seed=123)

als = ALS(maxIter=5, rank=70, regParam=0.01, coldStartStrategy='drop', userCol='steamidIndex', itemCol='appidIndex', ratingCol=selected_rating_col, implicitPrefs=True)
als.setSeed(123)
model = als.fit(training)

predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol=selected_rating_col, predictionCol="prediction")
rmse = evaluator.evaluate(predictions)

23/03/31 20:49:59 WARN DAGScheduler: Broadcasting large task binary with size 2.4 MiB
23/03/31 20:49:59 WARN DAGScheduler: Broadcasting large task binary with size 2.4 MiB


23/03/31 20:50:00 WARN DAGScheduler: Broadcasting large task binary with size 2.4 MiB
23/03/31 20:50:00 WARN DAGScheduler: Broadcasting large task binary with size 2.4 MiB
23/03/31 20:50:00 WARN DAGScheduler: Broadcasting large task binary with size 2.4 MiB
23/03/31 20:50:00 WARN DAGScheduler: Broadcasting large task binary with size 2.4 MiB
23/03/31 20:50:00 WARN DAGScheduler: Broadcasting large task binary with size 2.4 MiB
23/03/31 20:50:00 WARN DAGScheduler: Broadcasting large task binary with size 2.4 MiB
23/03/31 20:50:00 WARN DAGScheduler: Broadcasting large task binary with size 2.5 MiB
23/03/31 20:50:00 WARN DAGScheduler: Broadcasting large task binary with size 2.4 MiB
23/03/31 20:50:00 WARN DAGScheduler: Broadcasting large task binary with size 2.5 MiB


23/03/31 20:50:01 WARN DAGScheduler: Broadcasting large task binary with size 2.5 MiB
23/03/31 20:50:01 WARN DAGScheduler: Broadcasting large task binary with size 2.5 MiB
23/03/31 20:50:01 WARN DAGScheduler: Broadcasting large task binary with size 2.5 MiB
23/03/31 20:50:01 WARN DAGScheduler: Broadcasting large task binary with size 2.5 MiB
23/03/31 20:50:02 WARN DAGScheduler: Broadcasting large task binary with size 2.5 MiB
23/03/31 20:50:02 WARN DAGScheduler: Broadcasting large task binary with size 2.5 MiB
23/03/31 20:50:02 WARN DAGScheduler: Broadcasting large task binary with size 2.5 MiB
23/03/31 20:50:02 WARN DAGScheduler: Broadcasting large task binary with size 2.6 MiB
23/03/31 20:50:03 WARN DAGScheduler: Broadcasting large task binary with size 2.5 MiB
23/03/31 20:50:03 WARN DAGScheduler: Broadcasting large task binary with size 2.6 MiB
23/03/31 20:50:03 WARN DAGScheduler: Broadcasting large task binary with size 2.6 MiB
23/03/31 20:50:03 WARN DAGScheduler: Broadcasting larg

23/03/31 20:50:04 WARN DAGScheduler: Broadcasting large task binary with size 2.6 MiB
23/03/31 20:50:04 WARN DAGScheduler: Broadcasting large task binary with size 2.6 MiB
23/03/31 20:50:04 WARN DAGScheduler: Broadcasting large task binary with size 2.6 MiB
23/03/31 20:50:04 WARN DAGScheduler: Broadcasting large task binary with size 2.6 MiB
23/03/31 20:50:05 WARN DAGScheduler: Broadcasting large task binary with size 2.6 MiB
23/03/31 20:50:05 WARN DAGScheduler: Broadcasting large task binary with size 2.6 MiB
23/03/31 20:50:05 WARN DAGScheduler: Broadcasting large task binary with size 2.6 MiB


23/03/31 20:50:06 WARN DAGScheduler: Broadcasting large task binary with size 2.4 MiB
23/03/31 20:50:07 WARN DAGScheduler: Broadcasting large task binary with size 2.7 MiB
23/03/31 20:50:07 WARN DAGScheduler: Broadcasting large task binary with size 2.7 MiB


In [203]:
print(rmse)

1.7703203240582126
